In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
# in kernel to see where to load in data
#print(os.listdir("../input/"))
#load train and test data
#train = pd.read_csv('../input/train_V2.csv')
#test = pd.read_csv('../input/test_V2.csv')

train = pd.read_csv('train_V2.csv')
test = pd.read_csv('test_V2.csv')

In [3]:
#take a look at the training data
train.head()
print(train.shape)
#drop nan in train data
train = train.dropna()
print(train.shape)

(4446966, 29)
(4446965, 29)


In [4]:
#take a look at the features types
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4446965 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(4)
memory usage: 1017.8+ MB


In [5]:
#import xgboost
import xgboost

In [6]:
#set xgboost to regressor and set parameters
xgb = xgboost.XGBRegressor(learning_rate=0.05, gamma=0.3, max_depth=20, eval_metric = 'rmse', early_stopping_rounds=200)

In [7]:
#target
y_train = train['winPlacePerc']

In [8]:
y_train.head()

0    0.4444
1    0.6400
2    0.7755
3    0.1667
4    0.1875
Name: winPlacePerc, dtype: float64

In [9]:
#drop categorical features
X_train = train.drop(['Id', 'groupId', 'matchId','winPlacePerc', 'matchType' ], axis =1)
X_test = test.drop(['Id', 'groupId', 'matchId', 'matchType'], axis = 1)

In [10]:
#train model
xgb.fit(X_train, y_train)

[18:49:42] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=200, eval_metric='rmse',
       gamma=0.3, learning_rate=0.05, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [11]:
#predict the winplaceperc
pred = xgb.predict(X_test)
pred

array([0.2434369 , 0.91400254, 0.7720653 , ..., 0.89406633, 0.83988684,
       0.06825912], dtype=float32)

In [12]:
#save result to a csv
test_id = test["Id"]
submit_xg = pd.DataFrame({'Id': test_id, "winPlacePerc": pred} , columns=['Id', 'winPlacePerc'])
print(submit_xg.head())
submit_xg.to_csv("xgb_submission.csv", index = False)

               Id  winPlacePerc
0  9329eb41e215eb      0.243437
1  639bd0dcd7bda8      0.914003
2  63d5c8ef8dfe91      0.772065
3  cf5b81422591d1      0.527995
4  ee6a295187ba21      0.922543
